In [51]:
import time
import torch.nn as nn
from commun.CONSTANTE import *
import torch
from commun.CommunFonction import writeIntoFilesKeras
from commun.datasets.dataManager import DataManager
import torch.optim as optim
from math import ceil

Import data

In [52]:
beginTime = time.time()
data = DataManager("../commun/datasets/ANN/Churn_Modelling_traited.csv")


trainTensor,trainLabelTensor = data.getTrainCSV()
testTensor, testLabelTensor = data.getTestCSV()


Traitement data

In [53]:
trainTensor,trainLabelTensor = torch.tensor(trainTensor).float(),torch.tensor(trainLabelTensor).unsqueeze(1).float()
testTensor, testLabelTensor  = torch.tensor(testTensor).float(),torch.tensor(testLabelTensor).unsqueeze(1).float()


Create batch

In [54]:
def splitBatch(tensorToSplit,numberBatch):
    lstBatch = []
    sizeBatch = len(tensorToSplit)/numberBatch
    for i in range(numberBatch):
        supBorne = ceil(sizeBatch*(i+1))
        infBorne = ceil(sizeBatch*i)
        lstBatch.append(tensorToSplit[infBorne:supBorne][:])

    return lstBatch

trainBatch = splitBatch(trainTensor, BATCH_SIZE)
trainBatchLabel = splitBatch(trainLabelTensor,BATCH_SIZE)
testBatch = splitBatch(testTensor,BATCH_SIZE)
testBatchLabel = splitBatch(testLabelTensor,BATCH_SIZE)

Création modèle

In [55]:
model = nn.Sequential(
    nn.Linear(ANN_NUMBER_INPUT,NUMBER_NEURONE_HIDDEN_LAYER_ANN),
    nn.ReLU(),
    nn.Linear(NUMBER_NEURONE_HIDDEN_LAYER_ANN,NUMBER_NEURONE_HIDDEN_LAYER_ANN),
    nn.ReLU(),
    nn.Linear(NUMBER_NEURONE_HIDDEN_LAYER_ANN,NUMBER_NEURONE_HIDDEN_LAYER_ANN),
    nn.ReLU(),
    nn.Linear(NUMBER_NEURONE_HIDDEN_LAYER_ANN,NUMBER_NEURONE_OUTPUT),
    nn.Sigmoid()
)

optimizer = optim.Adam(model.parameters())

Training

In [56]:
def calculateAccuracy(result,target):
    goodPredictionTensor = (result > 0.5) == target
    total = len(goodPredictionTensor)
    goodPrediction = 0
    for boolean in goodPredictionTensor:
        if boolean:
            goodPrediction +=1
    return goodPrediction/total

def calculateMean(lstValue):
    total = 0
    for valueAcc in lstValue:
        total += valueAcc
    return total/len(lstValue)

lstAcc = []
lstValAcc = []
lstLoss = []
lstValLoss = []

def trainLoop(numberEpoch,optimizer,model,lossFunction,
              trainBatch,trainBatchLabel,
              testBatch,testBatchLabel):

    for epoch in range(1,numberEpoch+1):
        subLstAcc = []
        subLstAccVal = []
        subLstLoss = []
        subLstLossVal = []
        for train,trainLabel,test,testLabel in zip(trainBatch,trainBatchLabel,
                                                   testBatch,testBatchLabel):
            resultTrain = model(train)
            subLstAcc.append(calculateAccuracy(resultTrain,trainLabel))
            lossTrain = lossFunction(resultTrain,trainLabel)
            subLstLoss.append(lossTrain.item())
            with torch.no_grad(): #désactive l'accumulation pour mise a jour des poids et biais
                resultTest = model(test)
                subLstAccVal.append(calculateAccuracy(resultTest,testLabel))
                lossTest = lossFunction(resultTest,testLabel)
                subLstLossVal.append(lossTest.item())

            optimizer.zero_grad() #met a jour le buffer servant a l'accumulation
            # pour la mise a jour poids et biais(époque précédente)
            lossTrain.backward() #calcule le gradient des donnée
            optimizer.step() #met a jours les données 

        lstAcc.append(calculateMean(subLstAcc))
        lstValAcc.append(calculateMean(subLstAccVal))
        lstLoss.append(calculateMean(subLstLoss))
        lstValLoss.append(calculateMean(subLstLossVal))



In [57]:
trainLoop(NUMBER_EPOCH,optimizer,model,nn.BCELoss(),
          trainBatch,trainBatchLabel,
          testBatch, testBatchLabel)
endTime = time.time() - beginTime

print(lstAcc)
print(lstValAcc)
print(lstLoss)
print(lstValLoss)

[0.40225, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.7973750000000001, 0.798375, 0.8023750000000001, 0.8066250000000003, 0.8117500000000002]
[0.3976494495647723, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.7919306835637483, 0.79242671530978, 0.7969230030721968, 0.8004192268305174, 0.8024433563748081]
[0.7030991334468126, 0.6186714544892311, 0.518258847296238, 0.499027656391263, 0.491517161950469, 0.48356879968196154, 0.47475664503872395, 0.46960633248090744, 0.4659541817381978, 0.46260677743703127, 0.4591627214103937, 0.45532796531915665, 0.4510288508608937, 0.44637748412787914, 0.4417111035436392]
[0.7039840072393417, 0.6223315894603729, 0.5258154543116689, 0.5057820100337267, 0.49935947451740503, 0.4922151872888

In [58]:
writeIntoFilesKeras(lstAcc,lstValAcc,lstLoss,lstValLoss,endTime,"pytorch_ANN")
